In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import warnings

In [2]:
warnings.filterwarnings("ignore")

In [3]:
def get_content(number):
    url = f"https://sgschooling.com/year/{number}/all"
    soup = BeautifulSoup(
        requests.get(url).text, "html.parser"
    )
    table = soup.find("table")
    
    headings = [th.get_text().strip() for th in table.find("tr").find_all("th")]

    datasets = []
    
    for row in table.find_all("tr")[1:]:
        dataset = dict(zip(headings, (td.get_text() for td in row.find_all("td"))))
        dataset['Year'] = number
        datasets.append(dataset)

    return datasets

In [4]:
school_datasets = []

for number in range(2009,2023):
    data = get_content(number)
    school_datasets += data

In [5]:
df = pd.DataFrame(school_datasets)

In [6]:
#sort index
df = df.sort_index().reset_index(drop=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10044 entries, 0 to 10043
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   School   10044 non-null  object
 1   Phase 1  10044 non-null  object
 2   2A(1)    9320 non-null   object
 3   2A(2)    9320 non-null   object
 4   2B       10044 non-null  object
 5   2C       10044 non-null  object
 6   2C(S)    10044 non-null  object
 7   3        10044 non-null  object
 8   Year     10044 non-null  int64 
 9   2A       724 non-null    object
dtypes: int64(1), object(9)
memory usage: 784.8+ KB


In [8]:
# find the number of rows before 2019
length = len(df[df['Year'] < 2019])
length_before_2014 = len(df[df['Year'] < 2014])

length_before_2019 = len(df[df['Year'] < 2019])
length_before_2022 = len(df[df['Year'] < 2022])

print(length_before_2014)
print(length_before_2019)

3456
7112


In [9]:
# input vacancy at phase 1
df.iloc[1:length_before_2014:4,1] = df.iloc[1:length_before_2014:4,0].str.extract('(\d{3})')

df.iloc[3457:7113:4,1] = df.iloc[3457:7113:4,0].str.extract('(\d{3})').astype(int)-40

# input applied at phase 1 using taken figures
df.iloc[2:length:4,1] = df.iloc[3:length:4,1] 


In [10]:
#combine 2A(1) and 2A(2) into 2A
df.iloc[:length_before_2022,9] = df.iloc[:length_before_2022, 2].reset_index(drop=True).apply(pd.to_numeric, errors='coerce').fillna(0).astype(int) + df.iloc[:length_before_2022,3].reset_index(drop=True).apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

# drop the two columns
df.drop(['2A(1)', '2A(2)'], axis=1, inplace=True)

# rename phase 1 to 1
df.rename(columns={'Phase 1':'1'}, inplace=True)

# reorder the columns
df = df[['School', 'Year', '1', '2A', '2B', '2C', '2C(S)', '3']]

# input applied at 2A using taken figures
df.iloc[2:length_before_2019:4,3] = df.iloc[3:length_before_2019:4,3] 

# input Vacancy at 2A 

df.iloc[1:length_before_2022:4,3] = df.iloc[1:length_before_2022:4,2].reset_index(drop=True).apply(pd.to_numeric, errors='coerce').fillna(0).astype(int) - df.iloc[3:length_before_2022:4,2].reset_index(drop=True).apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10044 entries, 0 to 10043
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   School  10044 non-null  object
 1   Year    10044 non-null  int64 
 2   1       10044 non-null  object
 3   2A      10044 non-null  object
 4   2B      10044 non-null  object
 5   2C      10044 non-null  object
 6   2C(S)   10044 non-null  object
 7   3       10044 non-null  object
dtypes: int64(1), object(7)
memory usage: 627.9+ KB


In [12]:
melt_df = df.melt(id_vars=['School', 'Year'])

In [13]:
# Selects every 3rd row starting from 0
df2 = melt_df[melt_df.reset_index().index % 4 == 0].reset_index(drop=True).rename(columns={'variable':'Phase'})

df3 = df2.drop(['value'], axis=1)

df4 = pd.DataFrame(melt_df.iloc[1::4,3]).reset_index(drop=True).rename(columns={'value':'Vacancy'})

df5 = pd.DataFrame(melt_df.iloc[2::4,3]).reset_index(drop=True).rename(columns={'value':'Applied'})

df6 = pd.DataFrame(melt_df.iloc[3::4,3]).reset_index(drop=True).rename(columns={'value':'Taken'})

new_df = pd.concat([df3, df4, df5, df6], axis=1)


In [14]:
new_df

,School,Year,Phase,Vacancy,Applied,Taken
0,Admiralty,2009,1,300,168,168
1,Ahmad Ibrahim,2009,1,180,57,57
2,Ai Tong,2009,1,330,144,144
3,Anchor Green,2009,1,240,50,50
4,Anderson,2009,1,240,106,106
...,...,...,...,...,...,...
15061,Zhangde,2022,3,0,-,-
15062,Zhenghua,2022,3,0,-,-
15063,Zhonghua,2022,3,78,-,-
15064,Valour,2022,3,0,-,-


In [15]:
new_df.isnull().sum()

School     0
Year       0
Phase      0
Vacancy    0
Applied    0
Taken      0
dtype: int64

In [16]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15066 entries, 0 to 15065
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   School   15066 non-null  object
 1   Year     15066 non-null  int64 
 2   Phase    15066 non-null  object
 3   Vacancy  15066 non-null  object
 4   Applied  15066 non-null  object
 5   Taken    15066 non-null  object
dtypes: int64(1), object(5)
memory usage: 706.3+ KB


In [17]:
new_df['Balloting'] = new_df['Taken'].str.replace('(^\d*)', '')

In [18]:
new_df.isnull().sum()

School          0
Year            0
Phase           0
Vacancy         0
Applied         0
Taken           0
Balloting    2330
dtype: int64

In [19]:
new_df['Taken'] = new_df['Taken'].str.extract('(\d*)')

In [20]:
# pd.set_option('display.max_rows', None)
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15066 entries, 0 to 15065
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   School     15066 non-null  object
 1   Year       15066 non-null  int64 
 2   Phase      15066 non-null  object
 3   Vacancy    15066 non-null  object
 4   Applied    15066 non-null  object
 5   Taken      12736 non-null  object
 6   Balloting  12736 non-null  object
dtypes: int64(1), object(6)
memory usage: 824.0+ KB


In [21]:
new_df[new_df.isna().any(axis=1)]
new_df['Taken'] = new_df['Taken'].fillna(new_df['Applied'])

In [22]:
new_df['Balloting'] = new_df['Balloting'].fillna('')

In [23]:
new_df['Vacancy'] = new_df['Vacancy'].replace('-',0)
new_df['Vacancy'] = new_df['Vacancy'].replace('',0)

new_df['Vacancy'] = new_df['Vacancy'].astype(float).astype(int)

In [24]:
new_df['Applied'] = new_df['Applied'].replace('-',0)
new_df['Applied'] = new_df['Applied'].replace('',0)

new_df['Applied'] = new_df['Applied'].astype(float).astype(int)

In [25]:
new_df['Taken'] = new_df['Taken'].replace('-',0)
new_df['Taken'] = new_df['Taken'].replace('',0)

new_df['Taken'] = new_df['Taken'].astype(float).astype(int)

In [26]:
new_df['Application Rate'] = new_df['Applied'] / new_df['Vacancy']

In [27]:
new_df

,School,Year,Phase,Vacancy,Applied,Taken,Balloting,Application Rate
0,Admiralty,2009,1,300,168,168,,0.560000
1,Ahmad Ibrahim,2009,1,180,57,57,,0.316667
2,Ai Tong,2009,1,330,144,144,,0.436364
3,Anchor Green,2009,1,240,50,50,,0.208333
4,Anderson,2009,1,240,106,106,,0.441667
...,...,...,...,...,...,...,...,...
15061,Zhangde,2022,3,0,0,0,-,NaN
15062,Zhenghua,2022,3,0,0,0,-,NaN
15063,Zhonghua,2022,3,78,0,0,-,0.000000
15064,Valour,2022,3,0,0,0,-,NaN


In [28]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15066 entries, 0 to 15065
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   School            15066 non-null  object 
 1   Year              15066 non-null  int64  
 2   Phase             15066 non-null  object 
 3   Vacancy           15066 non-null  int32  
 4   Applied           15066 non-null  int32  
 5   Taken             15066 non-null  int32  
 6   Balloting         15066 non-null  object 
 7   Application Rate  12142 non-null  float64
dtypes: float64(1), int32(3), int64(1), object(3)
memory usage: 765.2+ KB


In [29]:
dict = {'Admiralty': 'Admiralty Primary School', 'Ahmad Ibrahim': 'Ahmad Ibrahim Primary School', 'Ai Tong': 'Ai Tong School', 'Alexandra': 'Alexandra Primary School', 'Anchor Green': 'Anchor Green Primary School', 'Anderson': 'Anderson Primary School', 'Anglo-Chinese (Junior)': 'Anglo-Chinese School (Junior)', 'Anglo-Chinese (Primary)': 'Anglo-Chinese School (Primary)', 'Angsana': 'Angsana Primary School', 'Ang Mo Kio': 'Ang Mo Kio Primary School', 'Balestier Hill': 'Balestier Hill Primary School', 'Beacon': 'Beacon Primary School', 'Bedok Green': 'Bedok Green Primary School', 'Bendemeer': 'Bendemeer Primary School', 'Blangah Rise': 'Blangah Rise Primary School', 'Boon Lay Garden': 'Boon Lay Garden Primary School', 'Bukit Panjang': 'Bukit Panjang Primary School', 'Bukit Timah': 'Bukit Timah Primary School', 'Bukit View': 'Bukit View Primary School', 'Canberra': 'Canberra Primary School', 'Canossa Catholic': 'Canossa Catholic Primary School', 'Cantonment': 'Cantonment Primary School', 'Casuarina': 'Casuarina Primary School', 'Catholic High': 'Catholic High School (Primary)', 'Cedar': 'Cedar Primary School', 'Changkat': 'Changkat Primary School', 'CHIJ (Katong)': 'CHIJ (Katong) Primary', 'CHIJ (Kellock)': 'CHIJ (Kellock)', 'CHIJ Our Lady of Good Counsel': 'CHIJ Our Lady of Good Counsel', 'CHIJ Our Lady of The Nativity': 'CHIJ Our Lady of the Nativity', 'CHIJ Our Lady Queen of Peace': 'CHIJ Our Lady Queen of Peace', 'CHIJ (Toa Payoh)': 'CHIJ Primary (Toa Payoh)', 'CHIJ St. Nicholas Girls’': "CHIJ St. Nicholas Girls' School", 'Chongfu': 'Chongfu School', 'Chongzheng': 'Chongzheng Primary School', 'Chua Chu Kang': 'Chua Chu Kang Primary School', 'Clementi': 'Clementi Primary School', 'Compassvale': 'Compassvale Primary School', 'Concord': 'Concord Primary School', 'Coral': 'Coral Primary School', 'Corporation': 'Corporation Primary School', 'Da Qiao': 'Da Qiao Primary School', 'Damai': 'Damai Primary School', 'Dazhong': 'Dazhong Primary School', 'De La Salle': 'De La Salle School', 'East Coast': 'East Coast Primary School', 'East Spring': 'East Spring Primary School', 'East View': 'East View Primary School', 'Edgefield': 'Edgefield Primary School', 'Elias Park': 'Elias Park Primary School', 'Endeavour': 'Endeavour Primary School', 'Eunos': 'Eunos Primary School', 'Evergreen': 'Evergreen Primary School', 'Fairfield Methodist': 'Fairfield Methodist School (Primary)', 'Farrer Park': 'Farrer Park Primary School', 'Fengshan': 'Fengshan Primary School', 'Fernvale': 'Fernvale Primary School', 'First Toa Payoh': 'First Toa Payoh Primary School', 'Frontier': 'Frontier Primary School', 'Fuchun': 'Fuchun Primary School', 'Fuhua': 'Fuhua Primary School', 'Gan Eng Seng': 'Gan Eng Seng Primary School', 'Geylang Methodist': 'Geylang Methodist School (Primary)', 'Gongshang': 'Gongshang Primary School', 'Greendale': 'Greendale Primary School', 'Greenridge': 'Greenridge Primary School', 'Greenwood': 'Greenwood Primary School', 'Guangyang': 'Guangyang Primary School', 'Haig Girls’': "Haig Girls' School", 'Holy Innocents’': "Holy Innocents' Primary School", 'Henry Park': 'Henry Park Primary School', 'Hong Wen': 'Hong Wen School', 'Horizon': 'Horizon Primary School', 'Hougang': 'Hougang Primary School', 'Huamin': 'Huamin Primary School', 'Innova': 'Innova Primary School', 'Jiemin': 'Jiemin Primary School', 'Jing Shan': 'Jing Shan Primary School', 'Junyuan': 'Junyuan Primary School', 'Jurong': 'Jurong Primary School', 'Jurong West': 'Jurong West Primary School', 'Juying': 'Juying Primary School', 'Keming': 'Keming Primary School', 'Kheng Cheng': 'Kheng Cheng School', 'Kong Hwa': 'Kong Hwa School', 'Kranji': 'Kranji Primary School', 'Kuo Chuan Presbyterian': 'Kuo Chuan Presbyterian Primary School', 'Lakeside': 'Lakeside Primary School', 'Lianhua': 'Lianhua Primary School', 'Loyang': 'Loyang Primary School', 'MacPherson': 'MacPherson Primary School', 'Maha Bodhi': 'Maha Bodhi School', 'Maris Stella High': 'Maris Stella High School', 'Marsiling': 'Marsiling Primary School', 'Marymount Convent': 'Marymount Convent School', 'Mayflower': 'Mayflower Primary School', 'Mee Toh': 'Mee Toh School', 'Meridian': 'Meridian Primary School', 'Methodist Girls’': "Methodist Girls' School (Primary)", 'Montfort Junior': 'Montfort Junior School', 'Nan Chiau': 'Nan Chiau Primary School', 'Nan Hua': 'Nan Hua Primary School', 'Nanyang': 'Nanyang Primary School', 'Ngee Ann': 'Ngee Ann Primary School', 'Naval Base': 'Naval Base Primary School', 'New Town': 'New Town Primary School', 'Northland': 'Northland Primary School', 'Northoaks': 'Northoaks Primary School', 'North Spring': 'North Spring Primary School', 'North View': 'North View Primary School', 'North Vista': 'North Vista Primary School', 'Oasis': 'Oasis Primary School', 'Opera Estate': 'Opera Estate Primary School', 'Palm View': 'Palm View Primary School', 'Park View': 'Park View Primary School', 'Pasir Ris': 'Pasir Ris Primary School', 'Paya Lebar Methodist Girls’': "Paya Lebar Methodist Girls' School (Primary)", 'Pei Chun Public': 'Pei Chun Public School', 'Pei Hwa Presbyterian': 'Pei Hwa Presbyterian Primary School', 'Pei Tong': 'Pei Tong Primary School', 'Peiying': 'Peiying Primary School', 'Pioneer': 'Pioneer Primary School', 'Poi Ching': 'Poi Ching School', 'Princess Elizabeth': 'Princess Elizabeth Primary School', 'Punggol Cove': 'Punggol Cove Primary School', 'Punggol Green': 'Punggol Green Primary School', 'Punggol': 'Punggol Primary School', 'Punggol View': 'Punggol View Primary School', 'Qifa': 'Qifa Primary School', 'Qihua': 'Qihua Primary School', 'Queenstown': 'Queenstown Primary School', 'Radin Mas': 'Radin Mas Primary School', 'Raffles Girls’': "Raffles Girls' Primary School", 'Red Swastika': 'Red Swastika School', 'Riverside': 'Riverside Primary School', 'River Valley': 'River Valley Primary School', 'Rivervale': 'Rivervale Primary School', 'Rosyth': 'Rosyth School', 'Rulang': 'Rulang Primary School', 'Sembawang': 'Sembawang Primary School', 'Sengkang Green': 'Sengkang Green Primary School', 'Seng Kang': 'Seng Kang Primary School', 'Shuqun': 'Shuqun Primary School', 'Si Ling': 'Si Ling Primary School', 'Singapore Chinese Girls’': "Singapore Chinese Girls' Primary School", 'South View': 'South View Primary School', 'Springdale': 'Springdale Primary School', 'Stamford': 'Stamford Primary School', 'St. Andrew’s Junior': "St. Andrew's Junior School", 'St. Anthony’s Canossian': "St. Anthony's Canossian Primary School", 'St. Anthony’s': "St. Anthony's Primary School", 'St. Gabriel’s': "St. Gabriel's Primary School", 'St. Hilda’s': "St. Hilda's Primary School", 'St. Joseph’s Institution Junior': "St. Joseph's Institution Junior", 'St. Margaret’s': "St. Margaret's Primary School", 'St. Stephen’s': "St. Stephen's School", 'Tampines North': 'Tampines North Primary School', 'Tampines': 'Tampines Primary School', 'Tanjong Katong': 'Tanjong Katong Primary School', 'Tao Nan': 'Tao Nan School', 'Teck Ghee': 'Teck Ghee Primary School', 'Teck Whye': 'Teck Whye Primary School', 'Telok Kurau': 'Telok Kurau Primary School', 'Temasek': 'Temasek Primary School', 'Townsville': 'Townsville Primary School', 'Unity': 'Unity Primary School', 'Waterway': 'Waterway Primary School', 'Wellington': 'Wellington Primary School', 'West Grove': 'West Grove Primary School', 'West Spring': 'West Spring Primary School', 'Westwood': 'Westwood Primary School', 'West View': 'West View Primary School', 'White Sands': 'White Sands Primary School', 'Woodgrove': 'Woodgrove Primary School', 'Woodlands': 'Woodlands Primary School', 'Woodlands Ring': 'Woodlands Ring Primary School', 'Xinghua': 'Xinghua Primary School', 'Xingnan': 'Xingnan Primary School', 'Xinmin': 'Xinmin Primary School', 'Xishan': 'Xishan Primary School', 'Yangzheng': 'Yangzheng Primary School', 'Yew Tee': 'Yew Tee Primary School', 'Yio Chu Kang': 'Yio Chu Kang Primary School', 'Yishun': 'Yishun Primary School', 'Yu Neng': 'Yu Neng Primary School', 'Yuhua': 'Yuhua Primary School', 'Yumin': 'Yumin Primary School', 'Zhangde': 'Zhangde Primary School', 'Zhenghua': 'Zhenghua Primary School', 'Zhonghua': 'Zhonghua Primary School', 'Fern Green': 'Fern Green Primary School', 'Northshore': 'Northshore Primary School', 'Valour': 'Valour Primary School'}

    
new_df = new_df.replace({"School": dict})

pd.DataFrame(new_df)

,School,Year,Phase,Vacancy,Applied,Taken,Balloting,Application Rate
0,Admiralty Primary School,2009,1,300,168,168,,0.560000
1,Ahmad Ibrahim Primary School,2009,1,180,57,57,,0.316667
2,Ai Tong School,2009,1,330,144,144,,0.436364
3,Anchor Green Primary School,2009,1,240,50,50,,0.208333
4,Anderson Primary School,2009,1,240,106,106,,0.441667
...,...,...,...,...,...,...,...,...
15061,Zhangde Primary School,2022,3,0,0,0,-,NaN
15062,Zhenghua Primary School,2022,3,0,0,0,-,NaN
15063,Zhonghua Primary School,2022,3,78,0,0,-,0.000000
15064,Valour Primary School,2022,3,0,0,0,-,NaN


In [30]:
new_df.to_csv('primary_school_clean.csv',index=False)